In [ ]:
!rm grc_files.sqlite

### Create SQLite DB

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [ ]:
df = pd.read_csv('grc_2019_file_index_extracts.csv')
data_dict = df.to_dict(orient='records')

In [ ]:
engine = create_engine('sqlite:///grc_files.sqlite')
conn = engine.connect()
Base = declarative_base()

In [ ]:
class Data(Base):
    __tablename__ = 'grc_2019'
    
    id = Column(Integer, primary_key=True)
    
    file_index = Column(Integer)
    file_name = Column(String)
    file_url = Column(String)
    file_text = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
Base.metadata.create_all(engine)
metadata = MetaData(bind=engine)
metadata.reflect()
table = sqlalchemy.Table('grc_2019', metadata, autoload=True)
conn.execute(table.delete())
conn.execute(table.insert(), data_dict)

### Query from SQLite DB

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
connection_string = 'sqlite:///grc_files.sqlite'
engine = create_engine(connection_string)
Base = automap_base()
Base.prepare(engine, reflect=True)
grc_2019 = Base.classes.grc_2019
session = Session(engine)
conn = engine.connect()
Base.classes.keys()

['grc_2019']

In [3]:
first_row = session.query(grc_2019).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1213c52d0>,
 'file_index': 2019000,
 'file_url': 'https://www.socalgas.com/regulatory/documents/a-17-10-008/SCG 2019 GRC Application-10-06-17-FINAL.PDF',
 'file_text': '[\'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'*NET OF ALLOWANCE FOR BORROWED FUNDS USED DURING CONSTRUCTION. ($7,117,529)\', \'1C6\', \'2C9\', \'3C9\', \'4C7\', \'1C5\', \'2C7\', \'3C6\', \'1C9\', \'2C5\', \'1C10\', \'2C5\', \'3C6\', \'4C10\', \'5C5\', \'6C11\', \'CALIFORNIA PUBLIC UTILITIES COMMISSIONService ListsParties DIANE CONKLIN                             EVELYN KAHL                              SPOKESPERSON                              COUNSEL                                  MUSSEY GRADE ROAD ALLIANCE

In [4]:
inspector = inspect(engine)
for each in inspector.get_columns('grc_2019'):
    print('Column Name: {} / Data Type: {}'.format(each['name'], each['type']))

Column Name: id / Data Type: INTEGER
Column Name: file_index / Data Type: INTEGER
Column Name: file_name / Data Type: VARCHAR
Column Name: file_url / Data Type: VARCHAR
Column Name: file_text / Data Type: TEXT


In [9]:
ansi_query = """
        select * 
        from grc_2019
        where file_index = 2019000
        """

In [10]:
df = pd.read_sql(ansi_query, conn)
df

,id,file_index,file_name,file_url,file_text
0,1,2019000,SCG 2019 GRC Application-10-06-17-FINAL.PDF,https://www.socalgas.com/regulatory/documents/...,"['(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)', '(a) ..."


In [8]:
all_file_idx = session.query(grc_2019.file_index).distinct().all()
all_file_idx[:10]

[(2019000),
 (2019001),
 (2019002),
 (2019003),
 (2019004),
 (2019005),
 (2019006),
 (2019007),
 (2019008),
 (2019009)]

In [28]:
TEMP = session.query(grc_2019.file_index, 
              grc_2019.file_name, 
             grc_2019.file_url,
             grc_2019.file_text).filter(grc_2019.file_index == 2019000).all()

In [32]:
temp_DICT = pd.DataFrame(TEMP).to_dict()

In [33]:
temp_DICT #RETURN THIS AS A JSON OBJECT VIA FLASK.

{'file_index': {0: 2019000},
 'file_name': {0: 'SCG 2019 GRC Application-10-06-17-FINAL.PDF'},
 'file_url': {0: 'https://www.socalgas.com/regulatory/documents/a-17-10-008/SCG 2019 GRC Application-10-06-17-FINAL.PDF'},
 'file_text': {0: '[\'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'(a) (b)(c) = (a) - (b)(d)(e)=(c)+(d)\', \'*NET OF ALLOWANCE FOR BORROWED FUNDS USED DURING CONSTRUCTION. ($7,117,529)\', \'1C6\', \'2C9\', \'3C9\', \'4C7\', \'1C5\', \'2C7\', \'3C6\', \'1C9\', \'2C5\', \'1C10\', \'2C5\', \'3C6\', \'4C10\', \'5C5\', \'6C11\', \'CALIFORNIA PUBLIC UTILITIES COMMISSIONService ListsParties DIANE CONKLIN                             EVELYN KAHL                              SPOKESPERSON                              COUNSEL                                  MUSSEY GRADE ROAD ALLI